In [1]:
import pandas as pd
import threading
import requests

In [2]:
# Initialize your data and DataFrame here
df = pd.read_excel("cXML Mappings - MAPPING_ANY_cXML_0000_InvoiceDetailRequest_1INV_0000_Invoice.xlsx")
new_df = df.copy()

In [3]:
# OpenAI access_token generation
def getAccessToken():


    client_id = "sb-7f966fb6-ddfb-4fbe-a0a6-402fc85bc3dc!b85851|azure-openai-service-i057149-xs!b16730"
    client_secret = "ba681a2e-e2f6-4ed7-b9ea-9536f7c239e7$C1aD40juyxlJg_eaEICYmMEJ9CYxY_kV2N_7Y-eAh7Q="
    uaa_url = "https://toolsteamai-2834lxx7.authentication.sap.hana.ondemand.com"

    params = {"grant_type": "client_credentials" }
    resp = requests.post(f"{uaa_url}/oauth/token",
                         auth=(client_id, client_secret),
                         params=params)

 

    token = resp.json()["access_token"]

 

    return token

In [4]:
input = new_df.to_string()

In [5]:
import re
def extract_code_from_response(generated_text):
    generated_text = str(generated_text)
    pattern = r"```xml([\s\S]*?)```"
    matches = re.finditer(pattern, generated_text, re.DOTALL)
    matches_list = [match.group(1) for match in matches]
    return matches_list

In [6]:
def generate_xml(new_df,i, chunks, token, svc_url, output, input):
    delimiter = "####"
    
    # Calculate the input range
    if (i + chunks) < len(new_df):
        input2 = new_df[i:i+chunks].to_string()
    else:
        input2 = new_df[i:len(new_df)].to_string()
        
    system_input= f"""



    You are given the contents of excel sheet in pandas dataframe string format which is mentioned inside the {delimiter}.



    The data is here {delimiter} {input} {delimiter} .When we input this string we get following xml output mentioned inside delimeter {delimiter}



The output is {delimiter} {output} {delimiter}. The taks is to generate the similar output xml for the sample input mentioned by user.The output displayed should only be xml in ```xml content here``` format, do generate for all rows 
"""

    data = {
        "deployment_id": "gpt-4-32k",
        "messages": [
            {"role": "system", "content": f"An interaction between a human and a machine. You have to do the following {system_input}"},
            {"role": "user", "content": f" kindly generate complete xml for this input {input2}, The output displayed should only be xml don't generate description "}
        ],
        "max_tokens": 1200,
        "temperature": 0.7,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "top_p": 0.95,
        "stop": "null"
    }

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    response = requests.post(f"{svc_url}/api/v1/completions", headers=headers, json=data)
    jsonContent = response.json()
    # print("___________________________",jsonContent['choices'][0]['message']['content'])
    print(extract_code_from_response(jsonContent['choices'][0]['message']['content']))
    with open(f"test{i}.xslt", "w") as file:
        file.write(extract_code_from_response(jsonContent['choices'][0]['message']['content'])[0])




In [7]:
# Initialize your data and DataFrame here
df = pd.read_excel("cXML Mappings - MAPPING_ANY_cXML_0000_InvoiceDetailRequest_1INV_0000_Invoice.xlsx")
new_df = df.copy()

# Read Invoice xslt file and store in output variable
with open("MAPPING_ANY_cXML_0000_InvoiceDetailRequest_1INV_0000_Invoice.xsl", "r") as file:
    output = file.read()

file.close()
    
chunks = 10  # Adjust the chunk size as needed
token = getAccessToken()
svc_url = "https://azure-openai-serv-i057149.cfapps.sap.hana.ondemand.com"

threads = []

i = 0
while i < len(df):
    print(i)
    thread = threading.Thread(target=generate_xml, args=(new_df,i, chunks, token, svc_url, output, input))
    threads.append(thread)
    thread.start()

    i += chunks

for thread in threads:
    thread.join()

print("All requests completed.")

0
10
20
30
40
50
60
70
80
['\n<!-- Create Party Section -->\n<xsl:template name="createParties">\n    <xsl:param name="v_path"/>\n    <invoiceParties>\n        <role>\n            <xsl:value-of select="substring(Contact/@role, 1, 30)"/>\n        </role>\n        <xsl:if test="string-length(normalize-space(Contact/Name)) > 0">\n            <name>\n                <xsl:value-of select="substring(Contact/Name, 1, 256)"/>\n            </name>\n            <xsl:if test="Contact/PostalAddress">\n                <providerAddresses>\n                    <xsl:for-each select="Contact/PostalAddress">\n                        <postalAddresses>\n                            <streetLines>\n                                <xsl:value-of select="Street"/>\n                            </streetLines>\n                            <town>\n                                <xsl:value-of select="City"/>\n                            </town>\n                            <xsl:if test="string-length(normalize-spac

In [30]:
# coding=utf8
# the above tag defines encoding for this document and is for Python 2.x compatibility

import re

regex = r"```xml(.*?)```"

test_str = ("Here is the xml output for your input:\\\\n\\\\n```xml\\\\n<?xml version=\\\\\\\"1.0\\\\\\\" encoding=\\\\\\\"UTF-8\\\\\\\"?>\\\\n<xsl:stylesheet xmlns:xsl=\\\\\\\"http://www.w3.org/1999/XSL/Transform\\\\\\\"\\\\n    xmlns:xs=\\\\\\\"http://www.w3.org/2001/XMLSchema\\\\\\\" version=\\\\\\\"2.0\\\\\\\" exclude-result-prefixes=\\\\\\\"#all\\\\\\\">\\\\n    <xsl:output method=\\\\\\\"xml\\\\\\\" indent=\\\\\\\"no\\\\\\\" omit-xml-declaration=\\\\\\\"yes\\\\\\\"/>\\\\n    <xsl:param name=\\\\\\\"anPayloadID\\\\\\\"/>\\\\n    <xsl:param name=\\\\\\\"documentreceiptdate\\\\\\\"/>\\\\n    <xsl:template match=\\\\\\\"Combined\\\\\\\">\\\\n        <root>\\\\n            <origin>\\\\n                <xsl:value-of select=\\\\\\\"'03'\\\\\\\"/>\\\\n            </origin>\\\\n            <supplierInvoiceId>\\\\n                <xsl:value-of select=\\\\\\\"substring(Payload/cXML/Request/InvoiceDetailRequest/InvoiceDetailRequestHeader/@invoiceID, 1,16)\\\\\\\"/>\\\\n            </supplierInvoiceId>\\\\n            <receiverSystemId>\\\\n                <xsl:value-of select=\\\\\\\"substring(Payload/cXML/Header/To/Credential[@domain='SystemID']/Identity, 1,30)\\\\\\\"/>\\\\n            </receiverSystemId>\\\\n            <receiverCompanyCode>\\\\n                <xsl:value-of select=\\\\\\\"substring(Payload/cXML/Request/InvoiceDetailRequest/InvoiceDetailRequestHeader/InvoicePartner/Contact[@role = 'billTo']/@addressID, 1, 10)\\\\\\\"/>\\\\n            </receiverCompanyCode>\\\\n            <receiverSupplier>\\\\n                <xsl:value-of select=\\\\\\\"substring(Payload/cXML/Header/From/Credential[@domain = 'VendorID']/Identity, 1,10)\\\\\\\"/>\\\\n            </receiverSupplier>\\\\n            <purpose>\\\\n                <xsl:value-of select=\\\\\\\"'I'\\\\\\\"/>\\\\n            </purpose>\\\\n            <currency>\\\\n                <xsl:value-of select=\\\\\\\"substring(Payload/cXML/Request/InvoiceDetailRequest/InvoiceDetailSummary/GrossAmount/Money/@currency, 1,5)\\\\\\\"/>\\\\n            </currency>\\\\n            <grossAmount>\\\\n                <xsl:value-of select=\\\\\\\"Payload/cXML/Request/InvoiceDetailRequest/InvoiceDetailSummary/GrossAmount/Money\\\\\\\"/>\\\\n            </grossAmount>\\\\n            <documentDate>\\\\n                <xsl:value-of select=\\\\\\\"substring(Payload/cXML/Request/InvoiceDetailRequest/InvoiceDetailRequestHeader/@invoiceDate, 1, 10)\\\\\\\"/>\\\\n            </documentDate>\\\\n            <documentHeaderText>\\\\n                <xsl:value-of select=\\\\\\\"substring(Payload/cXML/Request/InvoiceDetailRequest/InvoiceDetailRequestHeader/Comments, 1,25)\\\\\\\"/>\\\\n            </documentHeaderText>\\\\n        </root>\\\\n    </xsl:template>\\\\n</xsl:stylesheet>\\\\n```\n")

matches = re.finditer(regex, test_str, re.MULTILINE)

for matchNum, match in enumerate(matches, start=1):
    
    print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
    
    for groupNum in range(0, len(match.groups())):
        groupNum = groupNum + 1
        
        print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))

# Note: for Python 2.7 compatibility, use ur"" to prefix the regex and u"" to prefix the test string and substitution.


Match 1 was found at 44-2495: ```xml\\n<?xml version=\\\"1.0\\\" encoding=\\\"UTF-8\\\"?>\\n<xsl:stylesheet xmlns:xsl=\\\"http://www.w3.org/1999/XSL/Transform\\\"\\n    xmlns:xs=\\\"http://www.w3.org/2001/XMLSchema\\\" version=\\\"2.0\\\" exclude-result-prefixes=\\\"#all\\\">\\n    <xsl:output method=\\\"xml\\\" indent=\\\"no\\\" omit-xml-declaration=\\\"yes\\\"/>\\n    <xsl:param name=\\\"anPayloadID\\\"/>\\n    <xsl:param name=\\\"documentreceiptdate\\\"/>\\n    <xsl:template match=\\\"Combined\\\">\\n        <root>\\n            <origin>\\n                <xsl:value-of select=\\\"'03'\\\"/>\\n            </origin>\\n            <supplierInvoiceId>\\n                <xsl:value-of select=\\\"substring(Payload/cXML/Request/InvoiceDetailRequest/InvoiceDetailRequestHeader/@invoiceID, 1,16)\\\"/>\\n            </supplierInvoiceId>\\n            <receiverSystemId>\\n                <xsl:value-of select=\\\"substring(Payload/cXML/Header/To/Credential[@domain='SystemID']/Identity, 1,30)\\\"/

In [25]:
# Observations while extracting the code
''' 
1) The xml code can be extracted using regular expressions, with pattern regex = r"```xml(.*?)```"
2) There are 2 possible code generations; a) with xml as first tag b) direct templates to identify these
there is a simple logic check if there is a root in the code if yes go for a) else b).
'''

' \n1) The xml code can be extracted using regular expressions, with pattern regex = r"```xml(.*?)```"\n2) There are 2 possible code generations; a) with xml as first tag b) direct templates to identify these\nthere is a simple logic check if there is a root in the code if yes go for a) else b).\n'